In [ ]:
from __future__ import division, print_function

# Third-party
import astropy.units as u
import matplotlib.pyplot as pl
import numpy as np
pl.style.use('apw-notebook')
%matplotlib inline

# Custom
import gary.coordinates as gc
import gary.dynamics as gd
import gary.integrate as gi
import gary.potential as gp
from gary.units import galactic

import scf

In [ ]:
potential = gp.LogarithmicPotential(v_c=200*u.km/u.s, r_h=20*u.kpc, units=galactic,
                                    q1=1, q2=1, q3=1)

In [ ]:
w0 = gd.CartesianPhaseSpacePosition(pos=[10,0,0.]*u.kpc,
                                    vel=[0,0,150.]*u.km/u.s)
orbit = potential.integrate_orbit(w0, dt=-0.5, nsteps=8000)

fig = orbit.plot()

In [ ]:
pos = orbit[-1].pos.T.to(u.kpc).value[0]
vel = orbit[-1].vel.T.to(u.km/u.s).value[0]

cmd = ("python scripts/makerun.py --potentials='LogarithmicPotential' --dt=0.5 --nsteps=8000 "
       "--mass 200000 --name best-stream-data --path ../best-stream-data/sims/ "
       "--pos='{x[0]:.8f},{x[1]:.8f},{x[2]:.8f} kpc' "
       "--vel='{v[0]:.8f},{v[1]:.8f},{v[2]:.8f} km/s'").format(x=pos, v=vel)
print(cmd)

---

In [ ]:
rdr = scf.SCFReader("/Users/adrian/projects/best-stream-data/sims/best-stream-data/")

In [ ]:
tbl = rdr.read_snap('SNAP010', units=galactic)

In [ ]:
tbl

In [ ]:
pl.figure(figsize=(6,6))
pl.plot(tbl['x'], tbl['z'], ls='none')

w = orbit[orbit.t < (orbit.t.min()+2*u.Gyr)]
pl.plot(w.pos[0], w.pos[2], marker=None)

In [ ]:
nmax = 6
lmax = 3
PHI_NLM = np.random.random(size=(nmax+1, lmax+1, lmax+1))

In [ ]:
def getIndex3d(row, col, dep, ncol, ndep):
    return (row*ncol + col)*ndep + dep

for l in range(lmax+1):
    for m in range(l+1):
        for n in range(nmax+1):
            assert PHI_NLM[n,l,m] == PHI_NLM.ravel()[getIndex3d(n,l,m,lmax+1,lmax+1)]

In [ ]:
(0.2*u.kpc/u.Myr).to(u.km/u.s).value

---

In [ ]:
d0 = np.loadtxt("/Users/adrian/projects/scf/test/SNAP0000")

In [ ]:
for i in range(8+1):
    d = np.loadtxt("/Users/adrian/projects/scf/test/SNAP000{}".format(i))

    pl.figure(figsize=(5,5))
    pl.plot(d[:,0]/100., d[:,1]/100., ls='none', marker=',')
    pl.xlim(-20,20)
    pl.ylim(-20,20)
    pl.xlabel("$x$")
    pl.ylabel("$y$")

In [ ]:
# double rs = 10./config.ru;
# double vcirc2 = 220.*220./config.vu/config.vu;
# double GMs = 10.0*vcirc2/config.ru;
from astropy.constants import G

In [ ]:
rs = 10.*u.kpc
M = ((220.*u.km/u.s)**2 * rs / G).to(u.Msun)

potential = gp.HernquistPotential(m=M, c=rs, units=galactic)
w0 = gd.CartesianPhaseSpacePosition(pos=[15,0,0]*u.kpc,
                                    vel=[0,100,0.]*u.km/u.s)
w = potential.integrate_orbit(w0, dt=1., nsteps=4096)

In [ ]:
for i in range(8+1)[:4]:
    d = np.loadtxt("/Users/adrian/projects/scf/test/SNAP000{}".format(i))

    pl.figure(figsize=(5,5))
    pl.plot(d[:,0]/100., d[:,1]/100., ls='none', marker=',')
    pl.xlim(-20,20)
    pl.ylim(-20,20)
    pl.xlabel("$x$")
    pl.ylabel("$y$")
    
    pl.plot(w[:1500].pos[0], w[:1500].pos[1], marker=None, zorder=-100)

In [ ]:
d = np.loadtxt("/Users/adrian/projects/scf/test/SNAP0003")

pl.figure(figsize=(5,5))
pl.plot(d[:,0]/100., d[:,1]/100., ls='none', marker='.', alpha=0.1)
# pl.xlim(6,7)
# pl.ylim(-11.5,-10.5)
pl.xlabel("$x$")
pl.ylabel("$y$")

pl.plot(w[:1500].pos[0], w[:1500].pos[1], marker=None, zorder=-100)

In [ ]:
for i in range(8+1)[4:]:
    d = np.loadtxt("/Users/adrian/projects/scf/test/SNAP000{}".format(i))

    pl.figure(figsize=(5,5))
    pl.plot(d[:,0]/100., d[:,1]/100., ls='none', marker=',')
    pl.xlim(-20,20)
    pl.ylim(-20,20)
    pl.xlabel("$x$")
    pl.ylabel("$y$")
    
    pl.plot(w[1500:3500].pos[0], w[1500:3500].pos[1], marker=None, zorder=-100)

---


In [ ]:
import h5py
from gary.units import UnitSystem
import astropy.units as u
import matplotlib.pyplot as pl
pl.style.use('apw-notebook')
%matplotlib inline
import numpy as np

In [ ]:
with h5py.File("/Users/adrian/projects/scf/scripts/test.h5", 'r') as f:
    print(dict(f['units'].attrs))
    print(list(f['snapshots'].keys()))
    
    usys = UnitSystem([u.Unit(x) for x in f['units'].attrs.values()] + [u.radian])
    for i in range(3):
        print((f['snapshots/{}'.format(i)].attrs['t'] * usys['time']).to(u.Myr))
        
    pos = (f['snapshots/0/pos'][:]*usys['length']).to(u.kpc).value
    vel = (f['snapshots/0/vel'][:]*usys['length']/usys['time']).to(u.km/u.s).value
    
    cen_pos = (f['cen/pos'][:]*usys['length']).to(u.kpc).value
    cen_vel = (f['cen/vel'][:]*usys['length']/usys['time']).to(u.km/u.s).value

In [ ]:
pl.figure(figsize=(6,6))
pl.plot(pos[0], pos[1], ls='none')
pl.plot(cen_pos[0], cen_pos[1], marker=None)
pl.xlim(-20,20)
pl.ylim(-20,20)

In [ ]:
np.mean(pos, axis=-1)

In [ ]:
np.mean(vel, axis=-1)

In [ ]:
pl.figure(figsize=(6,6))
pl.plot(vel[0], vel[1], ls='none')
pl.plot(np.mean(vel[0]), np.mean(vel[1]), marker='o')
pl.xlim(-120,120)
pl.ylim(-120,120)